**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data files**

In [13]:
FOLDER_A = "annotation_fcc_table"
FD_REG_A = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER_A)

FOLDER_B = "annotation_ccres"
FD_REG_B = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER_B)

FD_REG_C = file.path(
    FD_REG_A,
    FOLDER_B
)

FD_OUT = file.path(
    FD_REG_A,
    "summary"
)

In [3]:
fdiry  = FD_REG_A
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "annotation_ccres"
[1] "annotation_ccres_silencer"
[1] "annotation_chromHMM"
[1] "annotation_tss_pol2"
[1] "description.tsv"
[1] "fcc_table.starrmpra.crispri.atac.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.concat.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.e2g_prediction.merge.tsv"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.e2g_benchmark.merge.tsv"
[1] "fcc_table.starrmpra.crispri.atac.merge.bed.gz"
[1] "fcc_table.starrmpra.crispri.atac.merge.tsv"
[1] "summary"


In [4]:
fdiry  = FD_REG_B
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "ccres_v3.bed.gz"
[1] "ccres_v4.bed.gz"
[1] "description.tsv"
[1] "summary"


In [5]:
fdiry  = FD_REG_C
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "peak.annotation.ccres_v3.bed.gz"
[1] "peak.annotation.ccres_v4.bed.gz"


## Import data

In [6]:
### set file paths
fdiry = FD_REG_A
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:3] = paste(vec[1:3], "A", sep="_")

### assign and show
vec_txt_cnames_reg_A = vec
print(vec)
dat

[1] "Chrom_A"     "Start_A"     "End_A"       "Assay_Type"  "Assay_Label"


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start
End,End
Assay_Type,Assay type
Assay_Label,Assay label


In [7]:
### set file paths
fdiry = FD_REG_B
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:6] = paste(vec[1:6], "B", sep="_")

### assign and show
vec_txt_cnames_reg_B = vec
print(vec)
dat

 [1] "Chrom_B"    "Start_B"    "End_B"      "Name_B"     "Score_B"   
 [6] "Strand_B"   "ThickStart" "ThickEnd"   "ItemRgb"    "Category"  
[11] "Note"      


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
ThickStart,The starting position at which the feature is drawn thickly
ThickEnd,The ending position at which the feature is drawn thickly
ItemRgb,"An RGB value of the form R,G,B (e.g. 255,0,0)"


In [8]:
### init: set column names
vec_txt_cnames = c(
    vec_txt_cnames_reg_A,
    vec_txt_cnames_reg_B,
    "Overlap"
)

### init: set file path
fdiry  = FD_REG_C
fname  = paste("peak.annotation", "bed.gz", sep="*")
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
fnames = basename(fpaths)

### init: annotation and label
annotations = fnames %>% 
    str_split(., "\\.") %>%
    do.call(rbind, .) %>%
    as.data.frame %>%
    dplyr::pull(3)
names(fnames)      = annotations
names(annotations) = annotations
print(fnames)

                         ccres_v3                          ccres_v4 
"peak.annotation.ccres_v3.bed.gz" "peak.annotation.ccres_v4.bed.gz" 


In [10]:
### Import data
lst = lapply(annotations, function(annotation){
    ### set file path
    fname = fnames[annotation]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = vec_txt_cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Annotation = annotation,
        Region_A   = paste0(Chrom_A, ":", Start_A, "-", End_A),
        Region_B   = paste0(Chrom_B, ":", Start_B, "-", End_B),
        Index_B    = Name_B,
        Label_B    = Category
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccres_v3" "ccres_v4"


Chrom_A,Start_A,End_A,Assay_Type,Assay_Label,Chrom_B,Start_B,End_B,Name_B,Score_B,⋯,ThickEnd,ItemRgb,Category,Note,Overlap,Annotation,Region_A,Region_B,Index_B,Label_B
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,180982,182087,"ASTARR,ATAC","ASTARR_R,ATAC",chr1,181251,181601,EH38E1310153,0,⋯,181601,"6,218,147",DNase-only,All-data/Full-classification,350,ccres_v3,chr1:180982-182087,chr1:181251-181601,EH38E1310153,DNase-only
chr1,777949,779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",chr1,778562,778912,EH38E1310158,0,⋯,778912,"255,0,0","PLS,CTCF-bound",All-data/Full-classification,350,ccres_v3,chr1:777949-779437,chr1:778562-778912,EH38E1310158,"PLS,CTCF-bound"
chr1,777949,779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",chr1,779086,779355,EH38E1310159,0,⋯,779355,"255,0,0",PLS,All-data/Full-classification,269,ccres_v3,chr1:777949-779437,chr1:779086-779355,EH38E1310159,PLS
chr1,816774,817547,ATAC,ATAC,chr1,817080,817403,EH38E1310166,0,⋯,817403,"225,225,225",Low-DNase,All-data/Full-classification,323,ccres_v3,chr1:816774-817547,chr1:817080-817403,EH38E1310166,Low-DNase
chr1,817905,818348,ATAC,ATAC,chr1,817903,818252,EH38E1310167,0,⋯,818252,"225,225,225",Low-DNase,All-data/Full-classification,347,ccres_v3,chr1:817905-818348,chr1:817903-818252,EH38E1310167,Low-DNase
chr1,818602,819380,"ATAC,WSTARR","ATAC,WSTARR_A",chr1,818718,818872,EH38E1310168,0,⋯,818872,"225,225,225",Low-DNase,All-data/Full-classification,154,ccres_v3,chr1:818602-819380,chr1:818718-818872,EH38E1310168,Low-DNase


## Arrange the columns

In [11]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### Select
    dat = dat %>%
        dplyr::select(
            Chrom_A, Start_A, End_A, Region_A, 
            Assay_Type, Assay_Label,
            Annotation,
            Chrom_B, Start_B, End_B, Region_B,
            Index_B,
            Label_B)

    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_A,  Start_A,  End_A)
    return(dat)
})

### assign and show
lst_peak_annot_arrange = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccres_v3" "ccres_v4"


Chrom_A,Start_A,End_A,Region_A,Assay_Type,Assay_Label,Annotation,Chrom_B,Start_B,End_B,Region_B,Index_B,Label_B
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
chr1,180982,182087,chr1:180982-182087,"ASTARR,ATAC","ASTARR_R,ATAC",ccres_v3,chr1,181251,181601,chr1:181251-181601,EH38E1310153,DNase-only
chr1,777949,779437,chr1:777949-779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",ccres_v3,chr1,778562,778912,chr1:778562-778912,EH38E1310158,"PLS,CTCF-bound"
chr1,777949,779437,chr1:777949-779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",ccres_v3,chr1,779086,779355,chr1:779086-779355,EH38E1310159,PLS
chr1,816774,817547,chr1:816774-817547,ATAC,ATAC,ccres_v3,chr1,817080,817403,chr1:817080-817403,EH38E1310166,Low-DNase
chr1,817905,818348,chr1:817905-818348,ATAC,ATAC,ccres_v3,chr1,817903,818252,chr1:817903-818252,EH38E1310167,Low-DNase
chr1,818602,819380,chr1:818602-819380,"ATAC,WSTARR","ATAC,WSTARR_A",ccres_v3,chr1,818718,818872,chr1:818718-818872,EH38E1310168,Low-DNase


## Save results

In [14]:
FD_OUT

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_fcc_table/summary"

In [15]:
lst = lst_peak_annot_arrange
for (idn in names(lst)){
    ### set file path
    fdiry = FD_OUT
    fname = paste("peak.annotation", idn, "tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()

    ### write table
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.annotation.ccres_v3.tsv"
[1] "peak.annotation.ccres_v4.tsv"


In [11]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### summarize
    dat = dat %>%
        dplyr::group_by(
            Chrom_A, Start_A, End_A, Region_A, Assay_Type, Assay_Label, 
            Annotation, 
            Label_B) %>%
        dplyr::summarise(
            Count    = n(), 
            Region_B = paste(Region, collapse="|"),
            
            Score_B  = mean(Value),
            Note   = paste(Note, collapse="|"),
            .groups = "drop")
    
    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_Peak,  Start_Peak,  End_Peak) %>%
        dplyr::select(-Chrom_Peak, -Start_Peak, -End_Peak) %>%
        dplyr::rename("Peak" = Name_Peak)
    return(dat)
})

### assign and show
lst_peak_annot_summary = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccres_v3" "ccres_v4"


Peak,Assay_Type,Assay_Label,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:180982-182087,"ASTARR,ATAC","ASTARR_R,ATAC",ccres_v3,DNase-only,1,chr1:181251-181601,0,EH38E1310153:DNase-only
chr1:777949-779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",ccres_v3,PLS,1,chr1:779086-779355,0,EH38E1310159:PLS
chr1:777949-779437,"ASTARR,ATAC,LMPRA,WSTARR","ASTARR_A,ATAC,LMPRA_A,WSTARR_A,WSTARR_AB",ccres_v3,"PLS,CTCF-bound",1,chr1:778562-778912,0,"EH38E1310158:PLS,CTCF-bound"
chr1:816774-817547,ATAC,ATAC,ccres_v3,Low-DNase,1,chr1:817080-817403,0,EH38E1310166:Low-DNase
chr1:817905-818348,ATAC,ATAC,ccres_v3,Low-DNase,1,chr1:817903-818252,0,EH38E1310167:Low-DNase
chr1:818602-819380,"ATAC,WSTARR","ATAC,WSTARR_A",ccres_v3,Low-DNase,1,chr1:818718-818872,0,EH38E1310168:Low-DNase


## Explore and check results

In [12]:
lst = lst_peak_annot_summary
lst = lapply(lst, function(dat){
    res = table(dat$Count)
    return(res)
})
print(lst)

$ccres_v3

     1      2      3      4      5      6      7      8      9     10     11 
123249  28267   8534   2781    968    377    133     53     15      6      3 

$ccres_v4

     1      2      3      4      5      6      7      8      9     10     11 
190961  48413  17663   6579   2533   1088    469    192     84     34     18 
    12     13     14 
     7      3      2 



## Save results

In [13]:
fdiry = file.path(
    FD_REG, 
    "summary")

lst = lst_peak_annot_summary
for (idn in names(lst)){
    fname = paste("peak.summary", idn, "tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()
    
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.summary.ccres_v3.tsv"
[1] "peak.summary.ccres_v4.tsv"
